Per eseguire i seguenti notebook, se non l'hai ancora fatto, devi impostare la chiave openai all'interno del file .env come `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Successivamente, caricheremo l'Embedding Index in un Dataframe di Pandas. L'Embedding Index è memorizzato in un file JSON chiamato `embedding_index_3m.json`. L'Embedding Index contiene gli Embedding per ciascuna delle trascrizioni YouTube fino alla fine di ottobre 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Successivamente, andremo a creare una funzione chiamata `get_videos` che cercherà la query nell’Embedding Index. La funzione restituirà i 5 video più simili alla query. Il funzionamento della funzione è il seguente:

1. Per prima cosa, viene creata una copia dell’Embedding Index.
2. Successivamente, viene calcolato l’Embedding per la query utilizzando l’API OpenAI Embedding.
3. Poi viene creata una nuova colonna nell’Embedding Index chiamata `similarity`. La colonna `similarity` contiene la similarità coseno tra l’Embedding della query e l’Embedding di ciascun segmento video.
4. In seguito, l’Embedding Index viene filtrato in base alla colonna `similarity`. L’Embedding Index viene filtrato per includere solo i video che hanno una similarità coseno maggiore o uguale a 0,75.
5. Infine, l’Embedding Index viene ordinato in base alla colonna `similarity` e vengono restituiti i primi 5 video.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Questa funzione è molto semplice, stampa solo i risultati della query di ricerca.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Per prima cosa, l’Embedding Index viene caricato in un Dataframe di Pandas.
2. Successivamente, all’utente viene chiesto di inserire una query.
3. Poi viene chiamata la funzione `get_videos` per cercare la query nell’Embedding Index.
4. Infine, viene chiamata la funzione `display_results` per mostrare i risultati all’utente.
5. All’utente viene quindi chiesto di inserire un’altra query. Questo processo continua finché l’utente non inserisce `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.it.png)

Ti verrà chiesto di inserire una query. Inserisci una query e premi invio. L’applicazione restituirà un elenco di video pertinenti alla query. L’applicazione fornirà anche un link al punto del video in cui si trova la risposta alla domanda.

Ecco alcune query che puoi provare:

- Che cos’è Azure Machine Learning?
- Come funzionano le reti neurali convoluzionali?
- Che cos’è una rete neurale?
- Posso usare i Jupyter Notebook con Azure Machine Learning?
- Che cos’è ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni di carattere critico, si raccomanda una traduzione professionale eseguita da un essere umano. Non siamo responsabili per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
